<img width="8%" alt="GitHub.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/GitHub.png" style="border-radius: 15%">

# GitHub - List issue comments
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=GitHub+-+List+issue+comments:+Error+short+description">Bug report</a>

**Tags:** #github #issue #comment #api #python #library

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-05-26 (Created: 2023-05-26)

**Description:** This notebook shows how to list comments from an issue on GitHub.

**References:**
- [GitHub API Documentation](https://docs.github.com/fr/rest/issues/comments?apiVersion=2022-11-28#list-issue-comments)

## Input

### Import libraries

In [ ]:
import requests
import naas
import pandas as pd

### Setup Variables
- `token`: [GitHub token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line)
- `owner`: owner of the repository
- `repo`: name of the repository
- `issue_number`: number of the issue

In [ ]:
token = naas.secret.get('GITHUB_TOKEN')
owner = "jupyter-naas"
repo = "awesome-notebooks"
issue_number = 1808

## Model

### Flatten nested dict

In [ ]:
# Flatten the nested dict
def flatten_dict(d, parent_key='', sep='_'):
    """
    Flattens a nested dictionary into a single level dictionary.

    Args:
        d (dict): A nested dictionary.
        parent_key (str): Optional string to prefix the keys with.
        sep (str): Optional separator to use between parent_key and child_key.

    Returns:
        dict: A flattened dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

### Create an issue comment

In [ ]:
def list_issue_comments(
    token=None,
    owner=None,
    repo=None,
    issue_number=None,
):
    # Init
    page = 1
    df = pd.DataFrame()
    while True:
        url = f"https://api.github.com/repos/{owner}/{repo}/issues/{issue_number}/comments?per_page=100&page={page}"
        response = requests.get(url)
        headers = {"Authorization": f"token {token}"}
        res = requests.get(url, headers=headers)
        if res.status_code == 200 and len(res.json()) > 0:
            res_json = res.json()
            for r in res_json:
                flatten_r = flatten_dict(r)
                tmp_df = pd.DataFrame([flatten_r])
                df = pd.concat([df, tmp_df]).sort_values("created_at")
            page += 1
        else:
            break
    return df

## Output

### Display result

In [ ]:
df = list_issue_comments(token, owner, repo, issue_number)
print('Comments fetched:', len(df))
print(df.info())
df